# Yolo v5

References: 
- https://github.com/ultralytics/yolov3/wiki/Train-Custom-Data
- https://colab.research.google.com/github/ultralytics/yolov3/blob/master/tutorial.ipynb#scrollTo=V0AJnSeCIHyJ
- https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb#scrollTo=1NcFxRcFdJ_O
- https://github.com/pylabel-project/pylabel

## Setup

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt --quiet

Cloning into 'yolov5'...
remote: Enumerating objects: 12557, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 12557 (delta 57), reused 65 (delta 30), pack-reused 12452
Receiving objects: 100% (12557/12557), 12.54 MiB | 7.01 MiB/s, done.
Resolving deltas: 100% (8593/8593), done.
/content/yolov5
     |████████████████████████████████| 596 kB 21.9 MB/s 


### Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import yaml
from pathlib import Path

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import torch

In [ ]:
# MODIFY HERE point to shared data folder
data_path = Path("/content/drive/MyDrive/Trabalho/Fruitpunch/AI for damage inspection/AI for Damage Inspection shared/01. Data/")

In [ ]:
local_data = Path("/content/dataset/")
local_data_path = local_data / "01. Data"

train_path = local_data_path / "train"
val_path = local_data_path / "val"

annotation_dir = "Annotations"
images_dir = "Images"

In [ ]:
DAMAGE_DICT = {'asr': 0, 'scheur': 1, 'betonrot': 2}

In [ ]:
train_path.mkdir(parents=True, exist_ok=True)
val_path.mkdir(parents=True, exist_ok=True)

## Copy files from drive to local dir

In [ ]:
!rm -rf /content/dataset

In [ ]:
!rsync -ah --info=progress2 "$data_path"/train "$local_data_path"

        568.46M 100%  860.38kB/s    0:10:45 (xfr#1433, to-chk=0/1436)


In [ ]:
!rsync -ah --info=progress2 "$data_path"/val "$local_data_path"

        142.87M 100%  797.77kB/s    0:02:54 (xfr#359, to-chk=0/362)


### Create .yaml file

In [ ]:
dict_file = {
        # train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
     'train': str(train_path / images_dir),
     'val': str(val_path / images_dir),
     'nc': len(DAMAGE_DICT), # number of classes
     'names': list(DAMAGE_DICT)
}

yaml_path = Path('/content/damage.yaml')

with open(yaml_path, 'w') as file:
    documents = yaml.dump(dict_file, file)

# Convert annotations from VOC to YOLOv5 format


In [ ]:
!pip install pylabel --quiet

     |████████████████████████████████| 367 kB 57.1 MB/s 


In [ ]:
"""
before importing modify this line (190)
    segmented = _GetValueOrBlank(root.find("segmented"))
to
    segmented = _GetValueOrBlank(root.find("segmented"), '')
from
    /usr/local/lib/python3.7/dist-packages/pylabel/importer.py
"""

from pylabel import importer

In [ ]:
dataset_train = importer.ImportVOC(
    path=train_path / annotation_dir, 
    path_to_images=train_path / images_dir,
    name="damage"
)

In [ ]:
dataset_val = importer.ImportVOC(
    path=val_path / annotation_dir, 
    path_to_images=val_path / images_dir,
    name="damage"
)

In [ ]:
dataset_train.df.head(5)

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,/content/dataset/01. Data/train/Images,P0079438_01_09.png,NaN,0,972,729,3,,2.0,4.0,...,NaN,NaN,Unspecified,0,0,0,asr,NaN,,1
1,/content/dataset/01. Data/train/Images,DSC03510_03_05.png,NaN,1,920,614,3,,1.0,4.0,...,NaN,NaN,Unspecified,0,0,1,scheur,NaN,,1
2,/content/dataset/01. Data/train/Images,P0079416_09_03.png,NaN,2,972,729,3,,18.0,10.0,...,NaN,NaN,Unspecified,0,0,0,asr,NaN,,1
3,/content/dataset/01. Data/train/Images,DSC03496_04_03.png,NaN,3,920,614,3,,1.0,1.0,...,NaN,NaN,Unspecified,0,0,1,scheur,NaN,,1
4,/content/dataset/01. Data/train/Images,P0079455_01_07.png,NaN,4,972,729,3,,6.0,2.0,...,NaN,NaN,Unspecified,0,0,0,asr,NaN,,1


In [ ]:
dataset_train.export.ExportToYoloV5(output_path=train_path / images_dir)

In [ ]:
dataset_val.export.ExportToYoloV5(output_path=val_path / images_dir)

# Train

In [ ]:
!pip install wandb --quiet

In [ ]:
!python train.py --img 640 --batch 16 --epochs 2000 --data $yaml_path --weights yolov5s.pt --cache

wandb: Currently logged in as: lupozz. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=, data=/content/damage.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=2000, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-376-g48a8531 Python-3.7.13 torch-1.12.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, o

# IN PROGRESS 

# Validate

In [ ]:
!python val.py --weights yolov3.pt --data coco.yaml --img 640 --iou 0.65 --half

# Test

In [ ]:
!python detect.py --weights yolov3.pt --img 640 --conf 0.25 --source data/images